# Implement GLM and Contrast Function
A generalized linear model is used to find where brain activity changes given different tasks
The contrast function compares the brain activity between target and general task.

We are interested in the following contrast:
- Differentiate brain region activity between target and general task during audio condition
- Differentiate brain region activity between target and general task during visual condition
- Differentiate brain region activity between audio and visual condition after contrast is applied to both conditions

In [17]:
# import dependencies
from nilearn.glm.first_level import FirstLevelModel
import numpy as np
import nibabel as nib
import nilearn.masking as masking
from nilearn.image import mean_img
import os
import glob
import pandas as pd
from nilearn.plotting import plot_design_matrix

In [18]:
# Categorize stimuli into target and general task
def categorize_stimuli(events_data, data_path):
    """ 
    Categorize stimuli based on the filename patterns in the events.tsv file
    For visual data: 'inh' = general task, 'sel' = target task
    events_data: pandas dataframe pulled from the events.tsv file
    data_path: path to the visual folder 
    """
    #extract event file
    print("base data_path: ", data_path)
    print("Input data shape: ", events_data.shape)
    print("Head of input data: ", events_data.head())

    pattern = os.path.join(data_path, 'sub-*/func/*events.tsv')
    event_files = glob.glob(pattern, recursive=True)
    if not event_files:
        raise ValueError(f"No events files found in {pattern}")

    # initialize empty list to store all events
    all_events = []

    for file_path in event_files:
        # load events file
        events = pd.read_csv(file_path, sep='\t')
        # categorize stimuli
        if 'inh' in file_path:
            events['condition'] = 'general'
        elif 'sel' in file_path:
            events['condition'] = 'target'      
        
        events['subject'] = os.path.basename(file_path).split('_')[0] # get the id of the subject
        all_events.append(events) # append the events to the list
    # verify categorization
    if events_data['condition'].isnull().all():
        raise ValueError("No condition column found in events data")
    
    # concatenate all events into a single dataframe
    categorized_events = pd.concat(all_events, ignore_index=True)

    # print summary
    print("\nData Summary:")
    print("Total number of events:", len(categorized_events))
    print("Events by condition:")
    print(categorized_events['condition'].value_counts())
    print("\nEvents by subject:")
    print(categorized_events['subject'].value_counts())
    
    return categorized_events

In [19]:
# Implement glm base model
def fit_glm(fmri_img, categorized_events, tr):
    """ 
    Fit a glm to the fmri data and the categorized events data
    """

    mask = masking.compute_epi_mask(
        fmri_img,
        lower_cutoff = 0.1,
        upper_cutoff = 0.9,
        connected = False,
        opening = False
    )

    model = FirstLevelModel(
        t_r=tr,
        mask_img = mask,
        standardize = True,
        minimize_memory = True,
    )
    
    glm = model.fit(fmri_img, events = categorized_events)
    
    return glm

In [20]:
# Implement contrast function
def contrast_glm_stimuli(glm, categorized_events, file_path, contrast_def = {'target': 1, 'general': -1}): 
    """ 
    Apply a contrast to the glm results 
    """
    events = categorize_stimuli(categorized_events, file_path)

    # convert dictionary to array
    conditions = sorted(contrast_def.keys()) # create a list of conditions
    contrast_vector = np.array([contrast_def[cond] for cond in conditions])

    print("Conditions: ", conditions) 
    print("Contrast vector: ", contrast_vector)

    contrast = glm.compute_contrast(contrast_vector)
    return contrast

def contrast_glm_conditions(glm, contrast_def = {'target': 1, 'general': -1}):
    """ 
    Apply a contrast to the glm results
    """
    conditions = sorted(contrast_def.keys()) # create a list of conditions
    contrast_vector = np.array([contrast_def[cond] for cond in conditions])
    
    contrast = glm.compute_contrast(contrast_vector)
    return contrast

In [21]:
# function to get design matrix
def create_design_matrix(glm):
    """ 
    Create a design matrix for the glm results
    """
     # get design matrix
    design_matrix = glm.design_matrices_[0]
    # set keys for the design matrix
    design_matrix.columns = ['constant', 'target', 'general', 'drift_1', 'drift_2', 'drift_3']
    
    return design_matrix


In [22]:
# Define parameters 
data_path = "../data/visual" # path to data folder
file_path = "../data/visual" # path to all individual data files
fmri_data = "../results/visual/cleaned_data_visual.nii.gz" #TODO: need to add audio data
fmri_img = nib.load(fmri_data)
tr = 1.5 # test with visual data

In [23]:
# function to run contrast and glm
def run_glm_contrast(fmri_img, data_path, tr):
    """ 
    Run the glm on the fmri data and apply contrast
    """
   
    categorized_events = categorize_stimuli(None, data_path)
    
    # Get GLM object 
    glm = fit_glm(fmri_img, categorized_events, tr)
    
    # Get design matrix
    design_matrix = glm.design_matrices_[0]
    design_matrix_columns = design_matrix.columns.tolist()
    print("columns: ", design_matrix_columns)
    design_matrix.columns = ['dummy', 'drift_1', 'drift_2', 'drift_3', 'drift_4', 'constant' ]
    
    # Compute contrasts
    contrast_stimuli = contrast_glm_stimuli(glm, categorized_events, data_path)
    contrast_conditions = contrast_glm_conditions(glm)
    
    return glm, design_matrix, contrast_stimuli, contrast_conditions

# Run GLM and get results
glm, design_matrix, contrast_stimuli, contrast_conditions = run_glm_contrast(fmri_img, data_path, tr)

# Plot design matrix (not the contrast)
plot_design_matrix(design_matrix)

base data_path:  ../data/visual


AttributeError: 'NoneType' object has no attribute 'shape'